## Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights, run_weighting_iteration


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(biden_vote, trump_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 5.0
    N_SIMS = 100000
    
    biden_moe = margin_of_error(n=n, p=biden_vote/100, interval_size=interval)
    trump_moe = margin_of_error(n=n, p=trump_vote/100, interval_size=interval)
    undecided = (100 - biden_vote - trump_vote) / 2

    biden_mean = biden_vote + undecided * 0.25
    biden_raw_moe = biden_moe * 100
    biden_allocate_undecided = undecided * 0.4
    biden_margin = biden_raw_moe + biden_allocate_undecided + GENERAL_POLLING_ERROR
    
    trump_mean = trump_vote + undecided * 0.25
    trump_raw_moe = trump_moe * 100
    trump_allocate_undecided = undecided * 0.4
    trump_margin = trump_raw_moe + trump_allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    biden_sigma = biden_margin / 100 / normed_sigma
    biden_sims = np.random.normal(biden_mean / 100, biden_sigma, N_SIMS)
    
    trump_sigma = trump_margin / 100 / normed_sigma
    trump_sims = np.random.normal(trump_mean / 100, trump_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(biden_sims, trump_sims)]) / N_SIMS
    
    low, high = np.percentile(biden_sims - trump_sims, [20, 80]) * 100
    
    return {'mean': biden_mean - trump_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': biden_raw_moe + trump_raw_moe,
            'margin': (biden_margin + trump_margin) / 2,
            'sigma': (biden_sigma + trump_sigma) / 2,
            'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Biden {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Biden {} likely to win)').format(mean,
                                                           first,
                                                           second,
                                                           n,
                                                           raw_moe,
                                                           margin,
                                                           sigma,
                                                           chance_pass))
    print(('Biden {} (80% CI: {} to {}) ({} Biden)').format(mean,
                                                            first,
                                                            second,
                                                            chance_pass))
    print('-')


def calc_result_sen(dem_vote, rep_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 5.0
    N_SIMS = 100000
    
    dem_moe = margin_of_error(n=n, p=dem_vote/100, interval_size=interval)
    rep_moe = margin_of_error(n=n, p=rep_vote/100, interval_size=interval)
    undecided = 100 - dem_vote - rep_vote

    dem_mean = dem_vote + undecided * 0.25
    dem_raw_moe = dem_moe * 100
    dem_allocate_undecided = undecided * 0.4
    dem_margin = dem_raw_moe + dem_allocate_undecided + GENERAL_POLLING_ERROR
    
    rep_mean = rep_vote + undecided * 0.25
    rep_raw_moe = rep_moe * 100
    rep_allocate_undecided = undecided * 0.4
    rep_margin = rep_raw_moe + rep_allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    dem_sigma = dem_margin / 100 / normed_sigma
    dem_sims = np.random.normal(dem_mean / 100, dem_sigma, N_SIMS)
    
    rep_sigma = rep_margin / 100 / normed_sigma
    rep_sims = np.random.normal(rep_mean / 100, rep_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(dem_sims, rep_sims)]) / N_SIMS
    
    low, high = np.percentile(dem_sims - rep_sims, [20, 80]) * 100
    
    return {'mean': dem_mean - rep_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': dem_raw_moe + rep_raw_moe,
            'margin': (dem_margin + rep_margin) / 2,
            'sigma': (dem_sigma + rep_sigma) / 2,
            'chance_pass': chance_pass}


def print_result_sen(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Dem Sen {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Dem Sen {} likely to win)').format(mean,
                                                             first,
                                                             second,
                                                             n,
                                                             raw_moe,
                                                             margin,
                                                             sigma,
                                                             chance_pass))
    print(('Dem {} (80% CI: {} to {}) ({} Dem)').format(mean,
                                                        first,
                                                        second,
                                                        chance_pass))

    print('-')

## Load Processed Data

In [2]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,40,41,42,43,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## State Presidential Models

In [3]:
POTUS_CENSUS = {'Alabama': {'Hillary Clinton': 0.3436, 'Donald Trump': 0.6208},
                'Alaska': {'Hillary Clinton': 0.3655, 'Donald Trump': 0.5128},
                'Arizona': {'Hillary Clinton': 0.4513, 'Donald Trump': 0.4867},
                'Arkansas': {'Hillary Clinton': 0.3365, 'Donald Trump': 0.6057},
                'California': {'Hillary Clinton': 0.6173, 'Donald Trump': 0.3162},
                'Colorado': {'Hillary Clinton': 0.4816, 'Donald Trump': 0.4325},
                'Connecticut': {'Hillary Clinton': 0.5457, 'Donald Trump': 0.4093},
                'Delaware': {'Hillary Clinton': 0.531, 'Donald Trump': 0.417},
                'Washington DC': {'Hillary Clinton': 0.905, 'Donald Trump': 0.016},
                'Florida': {'Hillary Clinton': 0.478, 'Donald Trump': 0.490},
                'Georgia': {'Hillary Clinton': 0.456, 'Donald Trump': 0.508},
                'Hawaii': {'Hillary Clinton': 0.622, 'Donald Trump': 0.300},
                'Idaho': {'Hillary Clinton': 0.275, 'Donald Trump': 0.593},
                'Illinois': {'Hillary Clinton': 0.558, 'Donald Trump': 0.379},
                'Indiana': {'Hillary Clinton': 0.379, 'Donald Trump': 0.511},
                'Iowa': {'Hillary Clinton': 0.417, 'Donald Trump': 0.512},
                'Kansas': {'Hillary Clinton': 0.361, 'Donald Trump': 0.567},
                'Kentucky': {'Hillary Clinton': 0.327, 'Donald Trump': 0.625},
                'Louisiana': {'Hillary Clinton': 0.385, 'Donald Trump': 0.581},
                'Maine': {'Hillary Clinton': 0.478, 'Donald Trump': 0.449},
                'Maryland': {'Hillary Clinton': 0.603, 'Donald Trump': 0.339},
                'Massachusetts': {'Hillary Clinton': 0.600, 'Donald Trump': 0.328},
                'Michigan': {'Hillary Clinton': 0.473, 'Donald Trump': 0.475},
                'Minnesota': {'Hillary Clinton': 0.464, 'Donald Trump': 0.449},
                'Mississippi': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Missouri': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Montana': {'Hillary Clinton': 0.381, 'Donald Trump': 0.562},
                'Nebraska': {'Hillary Clinton': 0.337, 'Donald Trump': 0.588},
                'Nevada': {'Hillary Clinton': 0.479, 'Donald Trump': 0.455},
                'New Hampshire': {'Hillary Clinton': 0.470, 'Donald Trump': 0.466},
                'New Jersey': {'Hillary Clinton': 0.555, 'Donald Trump': 0.414},
                'New Mexico': {'Hillary Clinton': 0.483, 'Donald Trump': 0.404},
                'New York': {'Hillary Clinton': 0.590, 'Donald Trump': 0.365},
                'North Carolina': {'Hillary Clinton': 0.462, 'Donald Trump': 0.498},
                'North Dakota': {'Hillary Clinton': 0.272, 'Donald Trump': 0.630},
                'Ohio': {'Hillary Clinton': 0.436, 'Donald Trump': 0.517},
                'Oklahoma': {'Hillary Clinton': 0.289, 'Donald Trump': 0.653},
                'Oregon': {'Hillary Clinton': 0.501, 'Donald Trump': 0.391},
                'Pennsylvania': {'Hillary Clinton': 0.475, 'Donald Trump': 0.481},
                'Rhode Island': {'Hillary Clinton': 0.544, 'Donald Trump': 0.389},
                'South Carolina': {'Hillary Clinton': 0.407, 'Donald Trump': 0.549},
                'South Dakota': {'Hillary Clinton': 0.317, 'Donald Trump': 0.615},
                'Tennessee': {'Hillary Clinton': 0.347, 'Donald Trump': 0.607},
                'Texas': {'Hillary Clinton': 0.432, 'Donald Trump': 0.522},
                'Utah': {'Hillary Clinton': 0.275, 'Donald Trump': 0.454},
                'Vermont': {'Hillary Clinton': 0.567, 'Donald Trump': 0.303},
                'Virginia': {'Hillary Clinton': 0.497, 'Donald Trump': 0.444},
                'Washington': {'Hillary Clinton': 0.525, 'Donald Trump': 0.368},
                'West Virginia': {'Hillary Clinton': 0.264, 'Donald Trump': 0.685},
                'Wisconsin': {'Hillary Clinton': 0.465, 'Donald Trump': 0.472},
                'Wyoming': {'Hillary Clinton': 0.216, 'Donald Trump': 0.674 }}

for state in POTUS_CENSUS.keys():
    print('## {} ##'.format(state.upper()))
    state_survey = survey.copy()
    potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
    potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
    potus_weights = output['weights']['vote2016']
    potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
    state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
    state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])

    options = ['Donald Trump', 'Hillary Clinton', 'Other']
    survey_ = state_survey.loc[state_survey['vote2016'].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    raw_result = potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    print('Raw result: {}'.format(np.round(raw_result * 100, 1)))
    print(votes)

    options = ['Joe Biden, the Democrat', 'Donald Trump, the Republican', 'Another candidate', 'Not decided']
    survey_ = state_survey.loc[state_survey['vote_trump_biden'].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

    votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                               trump_vote=votes['Donald Trump, the Republican'],
                               n=lv_weighted_n))
    print('-')

## ALABAMA ##
Raw result: -27.7
Donald Trump       62.429369
Hillary Clinton    33.968809
Other               3.601822
dtype: float64
Joe Biden, the Democrat         41.582205
Donald Trump, the Republican    51.858405
Another candidate                3.074509
Not decided                      3.484882
dtype: float64
-
Result Biden -10.3 (80% CI: -17.9 to -2.7) (Weighted N=1232) (raw_moe=3.6pts, margin=8.1pts, sigma=6.3pts) (Biden 12.7% likely to win)
Biden -10.3 (80% CI: -17.9 to -2.7) (12.7% Biden)
-
-
## ALASKA ##
Raw result: -14.7
Donald Trump       51.560631
Hillary Clinton    36.128300
Other              12.311069
dtype: float64
Joe Biden, the Democrat         45.297009
Donald Trump, the Republican    46.205581
Another candidate                4.814070
Not decided                      3.683340
dtype: float64
-
Result Biden -0.9 (80% CI: -8.8 to 7.0) (Weighted N=1285) (raw_moe=3.6pts, margin=8.5pts, sigma=6.6pts) (Biden 46.2% likely to win)
Biden -0.9 (80% CI: -8.8 to 7.0) (46.2% Bi

Raw result: -29.8
Donald Trump       62.829221
Hillary Clinton    32.316130
Other               4.854649
dtype: float64
Joe Biden, the Democrat         40.800152
Donald Trump, the Republican    52.378806
Another candidate                3.314828
Not decided                      3.506214
dtype: float64
-
Result Biden -11.6 (80% CI: -19.1 to -4.0) (Weighted N=1231) (raw_moe=3.6pts, margin=8.2pts, sigma=6.4pts) (Biden 9.9% likely to win)
Biden -11.6 (80% CI: -19.1 to -4.0) (9.9% Biden)
-
-
## LOUISIANA ##
Raw result: -19.6
Donald Trump       58.468724
Hillary Clinton    38.088875
Other               3.442400
dtype: float64
Joe Biden, the Democrat         44.302569
Donald Trump, the Republican    49.137135
Another candidate                3.063753
Not decided                      3.496543
dtype: float64
-
Result Biden -4.8 (80% CI: -12.4 to 2.6) (Weighted N=1258) (raw_moe=3.6pts, margin=8.1pts, sigma=6.3pts) (Biden 29.0% likely to win)
Biden -4.8 (80% CI: -12.4 to 2.6) (29.0% Biden)
-
-
##

Raw result: -35.8
Donald Trump       63.253341
Hillary Clinton    26.847371
Other               9.899288
dtype: float64
Joe Biden, the Democrat         38.445358
Donald Trump, the Republican    53.669237
Another candidate                4.288898
Not decided                      3.596507
dtype: float64
-
Result Biden -15.2 (80% CI: -23.0 to -7.5) (Weighted N=1213) (raw_moe=3.6pts, margin=8.4pts, sigma=6.5pts) (Biden 4.9% likely to win)
Biden -15.2 (80% CI: -23.0 to -7.5) (4.9% Biden)
-
-
## OHIO ##
Raw result: -8.1
Donald Trump       52.069172
Hillary Clinton    43.168460
Other               4.762368
dtype: float64
Joe Biden, the Democrat         48.049506
Donald Trump, the Republican    45.055049
Another candidate                3.351372
Not decided                      3.544073
dtype: float64
-
Result Biden 3.0 (80% CI: -4.5 to 10.5) (Weighted N=1296) (raw_moe=3.5pts, margin=8.2pts, sigma=6.4pts) (Biden 63.0% likely to win)
Biden 3.0 (80% CI: -4.5 to 10.5) (63.0% Biden)
-
-
## OKLAHOM

## State Models (Alt Weights, Post-Hoc)

In [5]:
for state in POTUS_CENSUS.keys():
    print('## {} ##'.format(state.upper()))
    state_survey = survey.copy()
    potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
    potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
    potus_weights = output['weights']['vote2016']
    potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
    state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
    state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])
    state_survey['lv_weight_alt'] = state_survey['lv_weight']
    state_survey.loc[(~state_survey['voted2016']) & (state_survey['vote_trump_biden'] == 'Donald Trump, the Republican'), 'lv_weight_alt'] *= 1.662
    state_survey['lv_weight_alt'] = normalize_weights(state_survey['lv_weight_alt'])
    options = ['Joe Biden, the Democrat', 'Donald Trump, the Republican', 'Another candidate', 'Not decided']
    survey_ = state_survey.loc[state_survey['vote_trump_biden'].isin(options)].copy()
    survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
    votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight_alt'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                               trump_vote=votes['Donald Trump, the Republican'],
                               n=lv_weighted_n))
    print('-')

## ALABAMA ##
Joe Biden, the Democrat         40.246172
Donald Trump, the Republican    53.405190
Another candidate                2.975725
Not decided                      3.372913
dtype: float64
-
Result Biden -13.2 (80% CI: -20.8 to -5.7) (Weighted N=1164) (raw_moe=3.7pts, margin=8.1pts, sigma=6.3pts) (Biden 7.1% likely to win)
Biden -13.2 (80% CI: -20.8 to -5.7) (7.1% Biden)
-
-
## ALASKA ##
Joe Biden, the Democrat         43.841908
Donald Trump, the Republican    47.933648
Another candidate                4.659425
Not decided                      3.565018
dtype: float64
-
Result Biden -4.1 (80% CI: -12.0 to 3.9) (Weighted N=1164) (raw_moe=3.7pts, margin=8.5pts, sigma=6.6pts) (Biden 33.2% likely to win)
Biden -4.1 (80% CI: -12.0 to 3.9) (33.2% Biden)
-
-
## ARIZONA ##
Joe Biden, the Democrat         47.873917
Donald Trump, the Republican    45.115358
Another candidate                3.543051
Not decided                      3.467674
dtype: float64
-
Result Biden 2.8 (80% CI: -4.9 t

Result Biden 22.9 (80% CI: 15.2 to 30.7) (Weighted N=1164) (raw_moe=3.6pts, margin=8.3pts, sigma=6.5pts) (Biden 99.4% likely to win)
Biden 22.9 (80% CI: 15.2 to 30.7) (99.4% Biden)
-
-
## MICHIGAN ##
Joe Biden, the Democrat         49.024569
Donald Trump, the Republican    44.162022
Another candidate                3.359460
Not decided                      3.453948
dtype: float64
-
Result Biden 4.9 (80% CI: -2.8 to 12.5) (Weighted N=1164) (raw_moe=3.7pts, margin=8.2pts, sigma=6.4pts) (Biden 70.2% likely to win)
Biden 4.9 (80% CI: -2.8 to 12.5) (70.2% Biden)
-
-
## MINNESOTA ##
Joe Biden, the Democrat         49.329112
Donald Trump, the Republican    43.109368
Another candidate                4.035745
Not decided                      3.525776
dtype: float64
-
Result Biden 6.2 (80% CI: -1.5 to 14.0) (Weighted N=1164) (raw_moe=3.7pts, margin=8.4pts, sigma=6.5pts) (Biden 75.2% likely to win)
Biden 6.2 (80% CI: -1.5 to 14.0) (75.2% Biden)
-
-
## MISSISSIPPI ##
Joe Biden, the Democrat       

Joe Biden, the Democrat         41.785279
Donald Trump, the Republican    46.856180
Another candidate                7.506740
Not decided                      3.851801
dtype: float64
-
Result Biden -5.1 (80% CI: -13.6 to 3.5) (Weighted N=1164) (raw_moe=3.7pts, margin=9.1pts, sigma=7.1pts) (Biden 30.8% likely to win)
Biden -5.1 (80% CI: -13.6 to 3.5) (30.8% Biden)
-
-
## VERMONT ##
Joe Biden, the Democrat         57.090863
Donald Trump, the Republican    34.328869
Another candidate                4.925535
Not decided                      3.654733
dtype: float64
-
Result Biden 22.8 (80% CI: 14.8 to 30.7) (Weighted N=1164) (raw_moe=3.6pts, margin=8.5pts, sigma=6.7pts) (Biden 99.2% likely to win)
Biden 22.8 (80% CI: 14.8 to 30.7) (99.2% Biden)
-
-
## VIRGINIA ##
Joe Biden, the Democrat         50.755732
Donald Trump, the Republican    42.258990
Another candidate                3.507785
Not decided                      3.477492
dtype: float64
-
Result Biden 8.5 (80% CI: 0.8 to 16.1) (Weight

## Senate Models

In [ ]:
SENATE_STATES = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Colorado', 'Delaware', 'Georgia',
                 'Idaho', 'Illinois', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
                 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Montana', 'Nebraska',
                 'New Hampshire', 'New Jersey', 'New Mexico', 'North Carolina', 'Oklahoma',
                 'Oregon', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
                 'Texas', 'Virginia', 'West Virginia', 'Wyoming']

for state in SENATE_STATES:
    print('## {} ##'.format(state.upper()))
    state_survey = survey.copy()
    potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
    potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
    potus_weights = output['weights']['vote2016']
    potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
    state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
    state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])

    options = ['A Democratic candidate', 'A Republican candidate', 'Another candidate', 'Not decided']
    survey_ = state_survey.loc[state_survey['vote_senate'].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

    votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                       rep_vote=votes['A Republican candidate'],
                                       n=lv_weighted_n))
    print('-')